In [2]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

import warnings
warnings.filterwarnings('ignore')

C:\Users\ROG STRIX\anaconda3\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
movie=pd.read_csv("C:\\Users\\ROG STRIX\\Desktop\\DataSets\\Netflix\\movies.csv")
rating=pd.read_csv("C:\\Users\\ROG STRIX\\Desktop\\DataSets\\Netflix\\ratings.csv")
tags=pd.read_csv("C:\\Users\\ROG STRIX\\Desktop\\DataSets\\Netflix\\tags.csv")

In [4]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movie.tail()

,movieId,title,genres
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)
62422,209171,Women of Devil's Island (1962),Action|Adventure|Drama


In [6]:
rating.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [7]:
#creating a saperate culumn for year 

movie['year']=movie.title.str.extract('(\(\d\d\d\d\))', expand=False )

# remocing the paranthesis 
movie['year']=movie.title.str.extract('(\d\d\d\d)', expand=False)

In [8]:
movie.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [9]:
# removing the year from title column
movie['title']=movie.title.str.replace('(\(\d\d\d\d\))','')
#applying the strip function to get rid of any ending whitespace
#movie['title']=movie['title'].str.strip()
movie.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [10]:
rating.groupby('movieId').rating.mean()
rating.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [11]:
df=movie.join(rating,lsuffix=' N',rsuffix=' K')
df

,movieId N,title,genres,year,userId,movieId K,rating,timestamp
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1,296,5.0,1147880044
1,2,Jumanji,Adventure|Children|Fantasy,1995,1,306,3.5,1147868817
2,3,Grumpier Old Men,Comedy|Romance,1995,1,307,5.0,1147868828
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,1,665,5.0,1147878820
4,5,Father of the Bride Part II,Comedy,1995,1,899,3.5,1147868510
...,...,...,...,...,...,...,...,...
62418,209157,We,Drama,2018,494,102903,3.5,1532809067
62419,209159,Window of the Soul,Documentary,2001,494,103042,3.0,1532809256
62420,209163,Bad Poems,Comedy|Drama,2018,494,103249,3.0,1532809116
62421,209169,A Girl Thing,(no genres listed),2001,494,103372,3.0,1532911673


In [12]:
#df=movie.join(rating,lsuffix='l')
#df 
#why join not used ??

In [13]:
df=df.drop(['movieId K','genres','year','timestamp'],axis=1)

In [14]:
# There will be lot of nan value in our new dataframe as it is highly unlikely that that ours users have seen almost all movies.
movie_users=df.pivot(index='movieId N', columns='userId',values='rating').fillna(0)
matrix_movies_users=csr_matrix(movie_users.values)
print(matrix_movies_users)

  (0, 0)	5.0
  (1, 0)	3.5
  (2, 0)	5.0
  (3, 0)	5.0
  (4, 0)	3.5
  (5, 0)	4.0
  (6, 0)	3.5
  (7, 0)	3.5
  (8, 0)	5.0
  (9, 0)	4.0
  (10, 0)	3.5
  (11, 0)	4.0
  (12, 0)	2.5
  (13, 0)	2.5
  (14, 0)	2.5
  (15, 0)	3.5
  (16, 0)	4.5
  (17, 0)	4.0
  (18, 0)	5.0
  (19, 0)	5.0
  (20, 0)	4.5
  (21, 0)	4.0
  (22, 0)	5.0
  (23, 0)	5.0
  (24, 0)	5.0
  :	:
  (62398, 493)	3.0
  (62399, 493)	3.5
  (62400, 493)	4.0
  (62401, 493)	3.5
  (62402, 493)	3.5
  (62403, 493)	2.5
  (62404, 493)	3.0
  (62405, 493)	3.5
  (62406, 493)	3.0
  (62407, 493)	0.5
  (62408, 493)	2.5
  (62409, 493)	2.0
  (62410, 493)	1.5
  (62411, 493)	3.5
  (62412, 493)	3.5
  (62413, 493)	3.0
  (62414, 493)	3.5
  (62415, 493)	3.5
  (62416, 493)	3.0
  (62417, 493)	3.5
  (62418, 493)	3.5
  (62419, 493)	3.0
  (62420, 493)	3.0
  (62421, 493)	3.0
  (62422, 493)	4.0


In [15]:
knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)

In [16]:
knn.fit(matrix_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [17]:
def recommender(movie_name, data,model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df['title'])[2]
    print('Movie Selected:-',df['title'][idx], 'Index: ',idx)
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df['title'][i].where(i!=idx))

In [18]:
recommender('Sudden Death', matrix_movies_users, knn,5)

Movie Selected:- Sudden Death  Index:  8
Searching for recommendations.....
64                         Bio-Dome 
49              Usual Suspects, The 
47                       Pocahontas 
45    How to Make an American Quilt 
50                   Guardian Angel 
Name: title, dtype: object
